# A simple matrix product state (MPS) emulator

In this notebook, we show a very simple implementation of a circuit emulator using a MPS representation of the quantum state.

We use a ``MPSState`` class to store the state in MPS form, namely to store the tensors of the MPS:
$$\psi_{b_1, \dots, b_n} = \mathrm{Tr} \left[ (A^{(1)})^{b_1} \dots (A^{(n)})^{b_n}\right]$$

with $A^{(i)}$ a $\chi \times 2 \times \chi$ tensor: $$(A^{(i)})^{b_i}_{k_i, k_{i+1}}$$ with $b_i = 0, 1$ and $k_i = 0\dots \chi-1$.

We recall that applying a single qubit gate $U$ to qubit $i$ corresponds to contracting the gate $U$ to tensor $A^{(i)}$:
$$(\tilde{A}^{(i)})^{b}_{k, k'}= \sum_{b'} U_{b, b'} (A^{(i)})^{b'}_{k, k'}.$$
The two-qubit case is detailed in the exercise sheet.

The main numpy function that you will need is the ``numpy.tensordot`` method. It allow to contract two tensors $A$ and $B$. If the ``axes`` parameter is not set, then, supposing we start from $A_{i, j, k}$ and $B_{l, m, n, p}$, the contraction is done on the last and first index of $A$ and $B$, respectively:

$$C_{i, j, m, n, p} = \sum_k A_{i, j, k} B_{k, m, n, p}.$$

One can also specify which indices to sum over. For instance, if ``axes = [(0, 2), (1, 2)]``,

$$C_{i, l, p} = \sum_{k, m} A_{k, i, m} B_{l, k, m, p}.$$

The ``numpy.transpose`` method will also be useful to reorder indices.

## A skeleton class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tools import Circuit, gate_dic

class MPSState:
    """class containing the MPS representation of a quantum state
    
    the MPS is stored as a list of tensors.
    
    Attributes:
        nqbits (int): number of qubits
        tensors (list<np.array>): list of tensors
        bond_dim (int): bond dimension
        
    Args:
        nqbits (int): the number of qubits
        bond_dim (int): the (maximal) bond dimension (chi)
    """
    
    def __init__(self, nqbits, bond_dim=2):
        """initialize the MPS to |0, 0, ...0>
        
        Args:
            nqbits (int): the number of qubits
            bond_dim (int): the (maximal) bond dimension (chi)
        """
        self.nqbits = nqbits
        self.tensors = [np.zeros((1, 2, 1), np.complex128) for _ in range(nqbits)]
        # state |0, 0, ..., 0>
        for qb in range(nqbits):
            # self.tensors[qb] = # ...
        self.bond_dim = bond_dim
        
    def apply(self, gate, qbits, param=None):
        """
        Apply a gate to the current state.
        
        Args:
            gate (string): name of gate
            qbits (list<int>): qubits on which it acts
            param (float, optional): parameter of gate (if applicable)
        
        """
        # ...
        if len(qbits) > 2: raise Exception(f"The gate acts on too many ({len(qbits)}) qubits")
        gate_ = gate_dic[gate] if param is None else gate_dic[gate](param)
        # gate_ is a (2, 2) or (4,4) array
        
        if len(qbits) == 1:
            # apply the gate
            # Hint: use tensordot function
            self.tensors[qbits[0]] = # ...
        else:
            # put qubits in right order
            if qbits[0] < qbits[1]:
                q1, q2 = qbits[0], qbits[1] 
            else:
                q1, q2 = qbits[1], qbits[0]
                gate_ = np.reshape(gate_, (2, 2, 2, 2))
                # swap q1 and q2
                gate_ = np.transpose(gate_, (1, 0, 3, 2))
                gate_ = np.reshape(gate_, (4, 4))
                
            if q1 != q2 - 1: 
                raise Exception("the gate must be applied on neighboring qubits")
                
            # merge the two neighboring tensors
            # Hint: tensordot
            
            # apply the gate
            # Hint: tensordot
            
            # now compress by SVD
            # ...
            
            # renormalize so that sum s^2 = 1
            # ...
            
            # discard unnecessary columns/rows of u and vh
            # ...
            
            self.tensors[q1] = # ...
            self.tensors[q2] = # ...
            
        
    def __str__(self):
        string = ""
        for qb in range(self.nqbits):
            string += str(self.tensors[qb].shape) + "\n"
        return string
        
        
    def to_vec(self):
        """contract MPS to dense vector"""
        # to do!


### A few tests

In [ ]:
# a couple of tests
psi = MPSState(2)
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 0, 0]))<1e-13)

psi.apply("H", [0])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 1, 0])/np.sqrt(2))<1e-13)

psi.apply("CNOT", [0, 1])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 0, 1])/np.sqrt(2))<1e-13)

psi.apply("CNOT", [0, 1])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 1, 0])/np.sqrt(2))<1e-13)

psi.apply("CNOT", [1, 0])
assert(np.linalg.norm(psi.to_vec() - np.array([1, 0, 1, 0])/np.sqrt(2))<1e-13)

psi.apply("RY", [1], np.pi/2)
assert(np.linalg.norm(psi.to_vec() - np.array([1, 1, 1, 1])/2)<1e-13)

## An emulator (QPU) based on a MPS representation.

In [ ]:
class MPSQPU:
    "Simulator based on MPS"
    def __init__(self, nqbits, bond_dim=2):
        self.state = MPSState(nqbits, bond_dim)
        
    def submit(self, circuit):
        """
        Args:
            circuit (Circuit): a quantum circuit to be simulated
        """
        assert(circuit.nqbits == self.state.nqbits)
        for gate_tuple in circuit.gates:
            self.state.apply(*gate_tuple)
            
        return self.state



In [ ]:
# test with a circuit
circ = Circuit(2, [("H", [0]), ("CNOT", [0, 1])])

qpu = MPSQPU(2)
res = qpu.submit(circ)

print(res.to_vec()) # what do you expect?

## With random circuits

Here we create a function that returns a random circuit, and compare the execution of this circuit on a perfect emulator (``StateVectorQPU``) and our MPS emulator.

In [ ]:
from state_vector_qpu import StateVectorQPU # a Schrödinger-style dense representation simulator
nqbits = 10

def random_circuit(nqbits, nlayers):
    # function to create a random circuits on nqbits with nlayers

circ = random_circuit(nqbits, 6)
qpu = StateVectorQPU(nqbits, gate_dic)
res = qpu.submit(circ)

bond_dim_list = [2, 4, 8, 16, 32, 64]

fid_list = []
for bond_dim in bond_dim_list:
    qpu = MPSQPU(nqbits, bond_dim=bond_dim)
    res_mps = qpu.submit(circ)

    # compute fidelity |<psi|psi_MPS>|^2
    # fid =  
    print(bond_dim, fid)
    fid_list.append(fid)
    
plt.plot(bond_dim_list, fid_list, '-o', lw=3)
plt.xlabel("Bond dimension")
plt.ylabel("Fidelity")
plt.grid();

## Going further

- compute the run time as a function of the bond dimension
- implement the measurement of an observable $H$ decomposed on the Pauli basis, $H = \sum_i \lambda_i P_i$.
- implement the canonicalization of the MPS
- run the emulator on circuits with 100 qubits!